In [1]:
import pandas as pd 
import numpy as np 
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import re
import time

In [ ]:
import pandas as pd

# 컬럼명 신경쓰고 상위xpath 프로그램

In [30]:
import time
import urllib.parse
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# 웹 드라이버 초기화
driver = webdriver.Chrome()

# 기업명 리스트
company_list = ['오비맥주', '사이벨 헬스', '와탭랩스', '21세기전파상', '씨드앤', '라텔앤드파트너즈', '중고나라', '픽소', '라텔앤드파트너즈', '인텔렉투스', '21세기전파상', '피나클 (FINAKLE)', '픽소', '올림플래닛', '써머스플랫폼', '21세기전파상', '코코네엠', '피나클 (FINAKLE)', '라이프시맨틱스', '디랙스', '디에스랩글로벌'
]  

# 데이터를 저장할 리스트 초기화
data = []

try:
    for company_name in company_list:
        formatted_keyword = urllib.parse.quote(company_name)  # 기업명을 URL 형식에 맞게 변환
        url = f"https://search.incruit.com/list/search.asp?col=company&kw={formatted_keyword}"
        
        # 검색 결과가 있는지 확인
        response = requests.get(url)
        if "검색결과가 없습니다" not in response.text:
            # URL로 이동
            driver.get(url)
            
            # 첫 번째 기업 링크를 찾기 위한 XPath
            first_company_xpath = './/div[1]/div/div[1]/span/a'
    
            # 첫 번째 기업 링크 클릭
            first_company_link = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, first_company_xpath))
            )
            first_company_link.click()
            
            # 타임슬립 추가: 페이지가 완전히 로드될 때까지 기다리기 위해 2초 대기
            time.sleep(2)
    
            # 새로 열린 페이지로 전환
            driver.switch_to.window(driver.window_handles[1])
            
            # 새로 열린 페이지에서 데이터 크롤링
            # 페이지 소스 가져오기
            page_source = driver.page_source
            
            # BeautifulSoup을 사용하여 파싱
            soup = BeautifulSoup(page_source, 'html.parser')
            
            try:
                # 기업명, 설립연도, 매출액, 사원수, 업종분류, 기업규모 추출
                table_li = soup.select_one('div.table_li')
                if table_li:
                    corp_name = table_li.select_one('strong:contains("기업명") + span').text.strip()
                    corp_year = table_li.select_one('strong:contains("설립일") + span').text.strip()  # 설립일을 설립연도로 변경
                    sales_money = table_li.select_one('strong:contains("매출액") + span').text.strip()
                    employee_count = table_li.select_one('strong:contains("사원수") + span').text.strip()
                    sector = table_li.select_one('strong:contains("업종") + span').text.strip()  # 업종을 업종분류로 변경
                    # 기업규모는 없는 경우도 있을 수 있으므로 예외 처리
                    try:
                        company_scale = table_li.select_one('strong:contains("기업규모") + span').text.strip()
                    except AttributeError:
                        company_scale = None
                    
                    # 데이터 추가
                    data.append({
                        '기업명': corp_name,
                        '설립연도': corp_year,
                        '매출액': sales_money,
                        '사원수': employee_count,
                        '업종분류': sector,
                        '기업규모': company_scale
                    })
                else:
                    print("테이블 요소를 찾을 수 없습니다:", driver.current_url)
                
                # 주소와 홈페이지 URL 추출
                myCompany_info = soup.select_one('ul.myCompany_info')
                if myCompany_info:
                    address = myCompany_info.select_one('li:contains("주소")').text.strip().split(':')[-1].strip()
                    homepage = myCompany_info.select_one('li:contains("홈페이지")').text.strip().split(':')[-1].strip()
                    # 데이터 추가
                    data[-1]['주소'] = address
                    data[-1]['기업 홈페이지 URL'] = homepage  # 변수 이름을 기업 홈페이지 URL로 변경
                else:
                    print("회사 정보를 찾을 수 없습니다:", driver.current_url)
            except Exception as e:
                print(f"기업 정보를 크롤링하는 도중 오류가 발생했습니다: {e}")
                
            # 새로 열린 페이지 닫기
            driver.close()

            # 다시 처음 페이지로 전환
            driver.switch_to.window(driver.window_handles[0])

except Exception as e:
    print(f"검색 및 데이터 수집 도중 오류가 발생했습니다: {e}")

finally:
    # 웹 드라이버 종료
    driver.quit()

# 수집된 데이터 출력
for item in data:
    print(item)


기업 정보를 크롤링하는 도중 오류가 발생했습니다: 'NoneType' object has no attribute 'text'
기업 정보를 크롤링하는 도중 오류가 발생했습니다: 'NoneType' object has no attribute 'text'
기업 정보를 크롤링하는 도중 오류가 발생했습니다: 'NoneType' object has no attribute 'text'
기업 정보를 크롤링하는 도중 오류가 발생했습니다: 'NoneType' object has no attribute 'text'
기업 정보를 크롤링하는 도중 오류가 발생했습니다: 'NoneType' object has no attribute 'text'
기업 정보를 크롤링하는 도중 오류가 발생했습니다: 'NoneType' object has no attribute 'text'
기업 정보를 크롤링하는 도중 오류가 발생했습니다: 'NoneType' object has no attribute 'text'
{'기업명': '오비맥주(주)', '설립연도': '1998년 5월 4일', '매출액': '1조 5,600억 7,020만원 (2022)', '사원수': '2,011 명 재직중', '업종분류': '음·식료품·식품가공', '기업규모': '중견기업', '주소': '서울특별시 강남구 영동대로 517 8층 (삼성동,아셈타워)지도보기', '기업 홈페이지 URL': '//www.ob.co.kr'}
{'기업명': '사이벨 헬스(주)', '설립연도': '2023년 3월 13일', '매출액': '117억원 (2022)', '사원수': '90 명 재직중', '업종분류': '소프트웨어·솔루션·ASP', '기업규모': '중소기업', '주소': '서울특별시 동대문구 회기로 117-3 (회기동) 서울바이오허브 산업지원동 403호지도보기', '기업 홈페이지 URL': '//www.sibelhealth.com'}
{'기업명': '(주)와탭랩스', '설립연도': '2015년 7월 1일', '매출액': '62억 1,141만원 (2022

In [31]:
import csv

# 기업 정보 데이터
company_data = [
    {'corp_name': '오비맥주(주)', 'corp_year': '1998년 5월 4일', 'sales_money': '1조 5,600억 7,020만원 (2022)', 'employee': '2,011 명 재직중', 'sector': '음·식료품·식품가공', 'address': '서울특별시 강남구 영동대로 517 8층 (삼성동,아셈타워)지도보기', 'corp_url': '//www.ob.co.kr'},
    {'corp_name': '사이벨 헬스(주)', 'corp_year': '2023년 3월 13일', 'sales_money': '117억원 (2022)', 'employee': '90 명 재직중', 'sector': '소프트웨어·솔루션·ASP', 'address': '서울특별시 동대문구 회기로 117-3 (회기동) 서울바이오허브 산업지원동 403호지도보기', 'corp_url': '//www.sibelhealth.com'},
    {'corp_name': '(주)와탭랩스', 'corp_year': '2015년 7월 1일', 'sales_money': '62억 1,141만원 (2022)', 'employee': '85 명 재직중', 'sector': '소프트웨어·솔루션·ASP', 'address': '서울특별시 서초구 서초대로77길 17 13층 1303호 (서초동,BLOCK77)지도보기', 'corp_url': '//www.whatap.io'},
    {'corp_name': '(주)21세기전파상', 'corp_year': '2019년 12월 5일', 'sales_money': '2억 5,520만원 (2022)', 'employee': '3 명 재직중', 'sector': '도매·상품중개'},
    {'corp_name': '(주)씨드앤', 'corp_year': '2015년 5월 21일', 'sales_money': '4억원 (2023)', 'employee': '24 명 재직중', 'sector': '소프트웨어·솔루션·ASP', 'address': '서울특별시 성동구 왕십리로 115 (성수동1가) 헤이그라운드 서울숲점 605,606,607호지도보기', 'corp_url': '//www.seedn.co.kr'},
    {'corp_name': '(주)라텔앤드파트너즈', 'corp_year': '2020년 7월 9일', 'sales_money': '33억 2,945만원 (2021)', 'employee': '36 명 재직중', 'sector': '방송·광고·영상·오디오'},
    {'corp_name': '주식회사 중고나라', 'corp_year': '2013년 12월 27일', 'sales_money': '101억 994만원 (2022)', 'employee': '149 명 재직중', 'sector': '소프트웨어·솔루션·ASP', 'address': '서울특별시 서초구 서초대로 301 (서초동,동익성봉빌딩)7층지도보기', 'corp_url': '//web.joongna.com'},
    {'corp_name': '(주)픽소', 'corp_year': '2018년 3월 15일', 'sales_money': '37억 1,142만원 (2022)', 'sector': '소프트웨어·솔루션·ASP', 'address': '서울특별시 마포구 월드컵북로6길 53 (연남동,카라빌딩)지도보기', 'corp_url': '//www.pixo.co'},
    {'corp_name': '(주)올림플래닛', 'corp_year': '2015년 1월 13일', 'sales_money': '109억 4,648만원 (2022)', 'employee': '70 명 재직중', 'sector': '소프트웨어·솔루션·ASP', 'address': '서울특별시 강남구 테헤란로 521 (삼성동,파르나스타워) 7층지도보기', 'corp_url': '//www.olimplanet.com/'},
    {'corp_name': '(주)써머스플랫폼', 'corp_year': '1999년 10월 15일', 'sales_money': '341억 1,932만원 (2022)', 'employee': '203 명 재직중', 'sector': '인터넷·포털·컨텐츠·게임', 'address': '서울특별시 금천구 가산디지털1로 186 (가산동,제이플라츠) 13층 (주)써머스플랫폼지도보기', 'corp_url': '//www.summerce.com/'},
    {'corp_name': '코코네엠(주)', 'corp_year': '2021년 12월 2일', 'sales_money': '94억 4,972만원 (2022)', 'employee': '190 명 재직중', 'sector': '인터넷·포털·컨텐츠·게임', 'address': '서울특별시 강남구 학동로9길 19 코코네M 사옥지도보기', 'corp_url': '//cocone-m.com/'},
    {'corp_name': '(주)피나클 (FINAKLE)', 'corp_year': '2020년 12월 23일', 'sales_money': '7,000만원 (2022)', 'employee': '7 명 재직중', 'sector': '소프트웨어·솔루션·ASP'},
    {'corp_name': '주식회사 라이프시맨틱스', 'corp_year': '2012년 9월 26일', 'sales_money': '28억 1,892만원 (2022)', 'employee': '68 명 재직중', 'sector': '소프트웨어·솔루션·ASP', 'address': '서울특별시 강남구 언주로 533 (역삼동) 라이프시맨틱스빌딩지도보기', 'corp_url': '//lifesemantics.kr'},
    {'corp_name': '(주)디랙스', 'corp_year': '2001년 8월 18일', 'sales_money': '650억 9,875만원 (2022)', 'employee': '187 명 재직중', 'sector': '제조업기타', 'address': '경기도 안양시 만안구 전파로62번길 26 (안양동)지도보기', 'corp_url': '//www.draxfit.com'},
    {'corp_name': '(주)디에스랩글로벌', 'corp_year': '2020년 1월 13일', 'sales_money': '6억 4,095만원 (2022)', 'employee': '22 명 재직중', 'sector': '소프트웨어·솔루션·ASP', 'address': '서울특별시 영등포구 의사당대로 83 5층 103호 (여의도동,오투타워)지도보기', 'corp_url': '//clickai.ai'}
    # 나머지 기업 정보도 이어서 추가해주세요
]

# CSV 파일로 저장
with open('company_info.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['corp_name', 'corp_year', 'sales_money', 'employee', 'sector', 'address', 'corp_url']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for company in company_data:
        writer.writerow(company)

print("CSV 파일이 성공적으로 저장되었습니다.")


CSV 파일이 성공적으로 저장되었습니다.
